In [1]:
""" This script tries only using the TuLiP toolbox to solve a multirobot path planning (MrPP) problem.
As opposed to multirobot path execution (MrPE) problem, predefined paths are not given
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [2]:
# Define the parameters & desired trajectories
num_robots = 2  # must be > 1
num_cells = 10
init = np.array([1, 10])
dest = np.array([8, 3])

In [3]:
# Define the env_vars based on the params % trajectories
env_vars = set()
env_vars_list = []
for r in range(1, num_robots + 1):
    env_vars_list.append([])
    for p in range(1, num_cells + 1):
        new_env_var = 's' + str(r) + 'a' + str(p)
        env_vars |= {new_env_var}
        env_vars_list[r - 1].append(new_env_var)

In [4]:
print(env_vars)

{'s2a4', 's2a6', 's1a9', 's1a5', 's2a9', 's2a1', 's1a8', 's1a4', 's2a7', 's2a5', 's2a3', 's2a10', 's2a8', 's1a3', 's1a6', 's1a2', 's2a2', 's1a7', 's1a1', 's1a10'}


In [5]:
print(env_vars_list)

[['s1a1', 's1a2', 's1a3', 's1a4', 's1a5', 's1a6', 's1a7', 's1a8', 's1a9', 's1a10'], ['s2a1', 's2a2', 's2a3', 's2a4', 's2a5', 's2a6', 's2a7', 's2a8', 's2a9', 's2a10']]


In [6]:
# Define the initial conditions
env_init = set()
for r in range(num_robots):
    env_init |= {env_vars_list[r][init[r] - 1]}

In [7]:
print(env_init)

{'s2a10', 's1a1'}


In [8]:
env_safe = set()
env_prog = set()
# Encode the constraint that each robot can only appear in one position
env_unique = set()
for r in range(1, num_robots + 1):
    for i in range(len(env_vars_list[r - 1])):
        new_env_unique = '( ' + env_vars_list[r - 1][i] + ' ) <-> ( '
        for j in range(len(env_vars_list[r - 1])):
            if j != i:
                new_env_unique += '!' + env_vars_list[r - 1][j] + ' && '
        new_env_unique = new_env_unique[:-3] + ')'
        env_unique |= {new_env_unique}
env_safe |= env_unique

In [9]:
print(env_unique)

{'( s2a4 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a5 && !s2a6 && !s2a7 && !s2a8 && !s2a9 && !s2a10 )', '( s1a10 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a5 && !s1a6 && !s1a7 && !s1a8 && !s1a9 )', '( s2a9 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 && !s2a10 )', '( s2a1 ) <-> ( !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 && !s2a9 && !s2a10 )', '( s1a6 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a5 && !s1a7 && !s1a8 && !s1a9 && !s1a10 )', '( s1a5 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a6 && !s1a7 && !s1a8 && !s1a9 && !s1a10 )', '( s2a6 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a7 && !s2a8 && !s2a9 && !s2a10 )', '( s1a7 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a5 && !s1a6 && !s1a8 && !s1a9 && !s1a10 )', '( s2a5 ) <-> ( !s2a1 && !s2a2 && !s2a3 && !s2a4 && !s2a6 && !s2a7 && !s2a8 && !s2a9 && !s2a10 )', '( s1a1 ) <-> ( !s1a2 && !s1a3 && !s1a4 && !s1a5 && !s1a6 && !s1a7 && !s1a8 && !s1a9 && !s1a10 )', '( s2a3 )

In [10]:
# Encode the transition system relations
# transition = np.array([[0, 2, 7, 5], [1, 3, 0, 0], [2, 4, 0, 6], [3, 0, 0, 8], [0, 0, 1, 0], [0, 0, 3, 0], [0, 0, 0, 1], [0, 0, 4, 0]])
# transition = np.array([[0, 2, 0, 0], [1, 0, 0, 0]])
# transition = np.array([[0, 2, 0, 0], [1, 3, 0, 0], [0, 2, 0, 0]])
transition = np.array([[0, 2, 0, 0], [1, 3, 0, 4], [2, 0, 0, 0], [0, 0, 2, 5], [9, 0, 4, 6], [0, 0, 5, 7], [8, 10, 6, 0], [0, 7, 0, 0], [0, 5, 4, 6], [7, 0, 0, 0]])
actions = ['up', 'down', 'left', 'right']
env_trans = set()
sys_bad_act = set()
# env_trans |= {'( s1a1 ) -> ( stop1 || down1 )'}
# env_trans |= {'( s1a2 ) -> ( stop1 || up1 )'}
for i in range(1, num_cells + 1):
    for r in range(num_robots):
        for d in range(4):
            if transition[i - 1][d] > 0:
                env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                            env_vars_list[r][i - 1] + ' || ' + env_vars_list[r][transition[i - 1][d] - 1] + ' )'}
                # env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                            # env_vars_list[r][transition[i - 1][d] - 1] + ' )'}
            else:
                sys_bad_act |= {'( ' + env_vars_list[r][i - 1] + ' ) -> ( !' + actions[d] + str(r + 1) + ' )'}
                # env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + actions[d] + str(r + 1) + ' ) -> X( ' +
                      #  env_vars_list[r][i - 1] + ' )'}
        env_trans |= {'( ' + env_vars_list[r][i - 1] + ' && ' + 'stop' + str(r + 1) + ' ) -> X( ' +
                      env_vars_list[r][i - 1] + ' )'}

env_safe |= env_trans

In [11]:
print(env_trans)

{'( s2a7 && stop2 ) -> X( s2a7 )', '( s1a2 && stop1 ) -> X( s1a2 )', '( s2a7 && up2 ) -> X( s2a7 || s2a8 )', '( s2a4 && stop2 ) -> X( s2a4 )', '( s2a7 && down2 ) -> X( s2a7 || s2a10 )', '( s2a6 && left2 ) -> X( s2a6 || s2a5 )', '( s1a8 && down1 ) -> X( s1a8 || s1a7 )', '( s2a8 && down2 ) -> X( s2a8 || s2a7 )', '( s2a6 && stop2 ) -> X( s2a6 )', '( s1a10 && up1 ) -> X( s1a10 || s1a7 )', '( s1a2 && right1 ) -> X( s1a2 || s1a4 )', '( s1a3 && up1 ) -> X( s1a3 || s1a2 )', '( s2a3 && stop2 ) -> X( s2a3 )', '( s2a2 && up2 ) -> X( s2a2 || s2a1 )', '( s2a4 && right2 ) -> X( s2a4 || s2a5 )', '( s2a5 && stop2 ) -> X( s2a5 )', '( s1a7 && down1 ) -> X( s1a7 || s1a10 )', '( s1a9 && right1 ) -> X( s1a9 || s1a6 )', '( s1a10 && stop1 ) -> X( s1a10 )', '( s2a10 && up2 ) -> X( s2a10 || s2a7 )', '( s2a4 && left2 ) -> X( s2a4 || s2a2 )', '( s1a7 && up1 ) -> X( s1a7 || s1a8 )', '( s1a6 && left1 ) -> X( s1a6 || s1a5 )', '( s2a8 && stop2 ) -> X( s2a8 )', '( s1a4 && left1 ) -> X( s1a4 || s1a2 )', '( s1a5 && rig

In [12]:
print(len(env_trans))

60


In [13]:
# Encode the constraint that every robot should stay after reaching its destination
#env_dest_stop = set()
sys_dest = set()
for r in range(num_robots):
    #env_dest_stop |= {'( ' + env_vars_list[r][dest[r] - 1] + ' ) -> X( ' + env_vars_list[r][dest[r] - 1] + ' )'}
    sys_dest |= {env_vars_list[r][dest[r] - 1]}
#env_safe |= env_dest_stop

In [14]:
# print(env_dest_stop)

In [15]:
print(sys_dest)

{'s2a3', 's1a8'}


In [16]:
# Encode the constraint that each robot should either not go or eventually not in the current cell
env_go = set()
for r in range(1, num_robots + 1):
    for i in range(num_cells):
        if i != dest[r - 1] - 1:
            env_go |= {'( !' + env_vars_list[r - 1][i] + ' || ' + 'stop' + str(r) + ' )'}
env_prog |= env_go

# given that I defined the  effect of "unavailable actions" to be staying in place, either we need 
# to update this progress property (because if i am at a cell where left is not possible and i keep
# saying left this progress property requires me to move eventually  which does not make sense)
# the other option is  to explicitly ban the "unavailable actions" in system safety spec

# THE ABOVE MENTION PROBLEM IS FIXED IN THIS VERSION

In [17]:
print(env_go)

{'( !s2a9 || stop2 )', '( !s2a4 || stop2 )', '( !s1a7 || stop1 )', '( !s2a2 || stop2 )', '( !s1a9 || stop1 )', '( !s1a3 || stop1 )', '( !s1a2 || stop1 )', '( !s2a5 || stop2 )', '( !s2a1 || stop2 )', '( !s2a7 || stop2 )', '( !s2a8 || stop2 )', '( !s1a10 || stop1 )', '( !s1a1 || stop1 )', '( !s1a6 || stop1 )', '( !s1a4 || stop1 )', '( !s2a10 || stop2 )', '( !s1a5 || stop1 )', '( !s2a6 || stop2 )'}


In [18]:
# Define sys_vars
sys_vars = set()
for r in range(1, num_robots + 1):
    sys_vars |= {'stop' + str(r)}
    sys_vars |= {'left' + str(r)}
    sys_vars |= {'right' + str(r)}
    sys_vars |= {'up' + str(r)}
    sys_vars |= {'down' + str(r)}

In [19]:
print(sys_vars)

{'right2', 'stop2', 'right1', 'up1', 'up2', 'left2', 'down2', 'left1', 'stop1', 'down1'}


In [20]:
# Define initial command. Assume nothing.
sys_init = set()

sys_safe = set()
sys_prog = set()
sys_prog |= sys_dest
#sys_safe |= env_dest_stop
# Encode the constraint that each time one action must be selected
sys_good_act = set()
for r in range(1, num_robots + 1):
    sys_good_act |= {'( left' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( right' + str(r) + ' ) <-> ( ' + '!left' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( up' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!left' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( down' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!left' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( stop' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!left' + str(r) + ' )'}
    
sys_safe |= sys_good_act
sys_safe |= sys_bad_act
print(sys_good_act)
print(sys_bad_act)

{'( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )', '( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )', '( down2 ) <-> ( !right2 && !up2 && !left2 && !stop2 )', '( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )', '( stop1 ) <-> ( !right1 && !up1 && !down1 && !left1 )', '( left2 ) <-> ( !right2 && !up2 && !down2 && !stop2 )', '( stop2 ) <-> ( !right2 && !up2 && !down2 && !left2 )', '( right2 ) <-> ( !left2 && !up2 && !down2 && !stop2 )', '( up2 ) <-> ( !right2 && !left2 && !down2 && !stop2 )', '( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )'}
{'( s2a10 ) -> ( !right2 )', '( s2a1 ) -> ( !left2 )', '( s2a1 ) -> ( !right2 )', '( s1a6 ) -> ( !up1 )', '( s2a3 ) -> ( !right2 )', '( s1a5 ) -> ( !down1 )', '( s2a5 ) -> ( !down2 )', '( s1a8 ) -> ( !right1 )', '( s1a9 ) -> ( !up1 )', '( s1a1 ) -> ( !left1 )', '( s1a2 ) -> ( !left1 )', '( s1a1 ) -> ( !up1 )', '( s1a8 ) -> ( !up1 )', '( s2a8 ) -> ( !left2 )', '( s1a7 ) -> ( !right1 )', '( s1a1 ) -> ( !right1 )', '( s2a2 ) -> 

In [21]:
# Encode the constraint that if ending point is reached, not go. And if ending point is not reached, always exist some
# robot that goes
sys_stop = set()
#new_sys_nonstop = '( '
for r in range(1, num_robots + 1):
    sys_stop |= {'( ' + env_vars_list[r - 1][dest[r - 1] - 1] + ' ) -> ( ' + 'stop' + str(r) + ' )'}
    #new_sys_nonstop += '( !' + env_vars_list[r - 1][dest[r - 1] - 1] + ' ) || '
#new_sys_nonstop = new_sys_nonstop[:-3] + ') -> ( '
#for r in range(1, num_robots + 1):
#    new_sys_nonstop += '!stop' + str(r) + ' || '
#new_sys_nonstop = new_sys_nonstop[:-3] + ')'
#sys_stop |= {new_sys_nonstop}
#sys_safe |= sys_stop
# we do NOT want the not stop part because when there are multiple robots, the robots might choose to stop to give way
# to other robots. synthesizer should be able to figure out (eventually) not choosing stop when not necessary.

In [22]:
print(sys_stop)

{'( s2a3 ) -> ( stop2 )', '( s1a8 ) -> ( stop1 )'}


In [23]:
# Encode the collision avoidance constraints
# No need if only one robot
sys_collision = set()
for i in range(num_cells):
    new_sys_collision = '!( '
    for r in range(num_robots):
        new_sys_collision += env_vars_list[r][i] + ' && '
    new_sys_collision = new_sys_collision[:-3] + ')'
    sys_collision |= {new_sys_collision}
sys_safe |= sys_collision

In [24]:
print(sys_collision)

{'!( s1a5 && s2a5 )', '!( s1a6 && s2a6 )', '!( s1a1 && s2a1 )', '!( s1a7 && s2a7 )', '!( s1a4 && s2a4 )', '!( s1a2 && s2a2 )', '!( s1a10 && s2a10 )', '!( s1a3 && s2a3 )', '!( s1a8 && s2a8 )', '!( s1a9 && s2a9 )'}


In [25]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\A \E'  # Moore initial condition synthesized too
specs.moore = False
specs.plus_one = False

In [26]:
print(specs)

((s2a10) && (s1a1) && [](( s1a2 && stop1 ) -> X( s1a2 )) && [](( s2a7 && up2 ) -> X( s2a7 || s2a8 )) && [](( s2a7 && down2 ) -> X( s2a7 || s2a10 )) && [](( s2a1 ) <-> ( !s2a2 && !s2a3 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 && !s2a9 && !s2a10 )) && [](( s2a8 && down2 ) -> X( s2a8 || s2a7 )) && [](( s1a5 ) <-> ( !s1a1 && !s1a2 && !s1a3 && !s1a4 && !s1a6 && !s1a7 && !s1a8 && !s1a9 && !s1a10 )) && [](( s1a2 && right1 ) -> X( s1a2 || s1a4 )) && [](( s2a3 ) <-> ( !s2a1 && !s2a2 && !s2a4 && !s2a5 && !s2a6 && !s2a7 && !s2a8 && !s2a9 && !s2a10 )) && [](( s2a3 && stop2 ) -> X( s2a3 )) && [](( s1a7 && down1 ) -> X( s1a7 || s1a10 )) && [](( s1a10 && stop1 ) -> X( s1a10 )) && [](( s1a9 && right1 ) -> X( s1a9 || s1a6 )) && [](( s1a7 && up1 ) -> X( s1a7 || s1a8 )) && [](( s1a6 && left1 ) -> X( s1a6 || s1a5 )) && [](( s2a8 && stop2 ) -> X( s2a8 )) && [](( s1a4 && left1 ) -> X( s1a4 || s1a2 )) && [](( s1a5 && right1 ) -> X( s1a5 || s1a6 )) && [](( s2a3 && up2 ) -> X( s2a3 || s2a2 )) && [](( s1a7 

In [ ]:
print('Start synthesis')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis


In [ ]:
machines.random_run(ctrl, N=20)

In [ ]:
print(len(env_unique))